In [139]:
import re
import os
import unicodedata
import urllib3
import zipfile
import shutil
import numpy as np
import pandas as pd
import re
import tensorflow as tf
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [140]:
tf.__version__

'2.7.0'

In [141]:
pip install Korpora


In [142]:
num_samples = 33000

In [143]:
from Korpora import Korpora
Korpora.corpus_list()
corpus = Korpora.load('open_subtitles')


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : TRAC (https://trac.edgewall.org/)
    Repository : http://opus.nlpl.eu/OpenSubtitles-v2018.php
    References :
        - P. Lison and J. Tiedemann, 2016, OpenSubtitles2016: Extracting Large Parallel Corpora
          from Movie and TV Subtitles. In Proceedings of the 10th International Conference on
          Language Resources and Evaluation (LREC 2016)

    This is a new collection of translated movie subtitles from http://www.opensubtitles.org/.

    [[ IMPORTANT ]]
    If you use the OpenSubtitle corpus: Please, add a link to http://www.opensubtitles.org/
    to your website and to your reports and publications produced with the data!
    I promised this when I got the data from the providers of that website!

 

In [150]:
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

In [151]:
def preprocess_sentence(sent):
  # 위에서 구현한 함수를 내부적으로 호출
  sent = unicode_to_ascii(sent.lower())

  # 단어와 구두점 사이에 공백을 만듭니다.
  # Ex) "he is a boy." => "he is a boy ."
  sent = re.sub(r"([?.!,¿])", r" \1", sent)

  # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환합니다.
  sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

  sent = re.sub(r"\s+", " ", sent)
  return sent

In [152]:
korean = corpus.train[0:60000].text
english = corpus.train[0:60000].pair
lines = pd.DataFrame({'src':korean, 'tar':english})
print(lines)


                                                     src                                                tar
0      폭설이 내리고 우박, 진눈깨비가 퍼부어도 눈보라가 몰아쳐도 강풍이 불고 비바람이 휘...  Through the snow and sleet and hail, through t...
1                   우리의 한결같은 심부름꾼 황새 아저씨 가는 길을 그 누가 막으랴!  ever faithful, ever true, nothing stops him, h...
2                                          황새 아저씨를 기다리세요        Look out for Mr Stork That persevering chap
3                                         찾아와 선물을 주실 거예요     He'll come along and drop a bundle in your lap
4                                     가난하든 부자이든 상관이 없답니다    You may be poor or rich It doesn't matter which
...                                                  ...                                                ...
59995  If you really want to do something for me, the...                   - Oh, God, I can't believe this.
59996                                     - I'm leaving.                                     - I'm leaving.
59997        I've assessed t

In [153]:
def load_preprocessed_data():
  encoder_input, decoder_input, decoder_target = [], [], []

  print(len(korean))

  for i in range(len(korean)):

    # # source 데이터 전처리
    src_line = [w for w in korean[i].split()]
    src_line = re.sub('-', '', src_line)

    # print(src_line)

    # # target 데이터 전처리
    tar_line = preprocess_sentence(english[i])
    tar_line_in = [w for w in ("<sos> " + tar_line).split()]
    tar_line_out = [w for w in (tar_line + " <eos>").split()]

    encoder_input.append(src_line)
    decoder_input.append(tar_line_in)
    decoder_target.append(tar_line_out)

    if i == num_samples - 1:
      break
  return encoder_input, decoder_input, decoder_target

In [154]:
# 인코딩 테스트
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"
print(preprocess_sentence(en_sent))
print(preprocess_sentence(fr_sent).encode('utf-8'))

have you had dinner ?
b'avez vous deja dine ?'


In [155]:
sents_en_in, sents_fra_in, sents_fra_out = load_preprocessed_data()

60000


TypeError: ignored

In [156]:
print(sents_en_in[:5])
print(sents_fra_in[:5])
print(sents_fra_out[:5])

[['폭설이', '내리고', '우박,', '진눈깨비가', '퍼부어도', '눈보라가', '몰아쳐도', '강풍이', '불고', '비바람이', '휘몰아쳐도'], ['우리의', '한결같은', '심부름꾼', '황새', '아저씨', '가는', '길을', '그', '누가', '막으랴!'], ['황새', '아저씨를', '기다리세요'], ['찾아와', '선물을', '주실', '거예요'], ['가난하든', '부자이든', '상관이', '없답니다']]
[['<sos>', 'through', 'the', 'snow', 'and', 'sleet', 'and', 'hail', 'through', 'the', 'blizzard', 'through', 'the', 'gales', 'through', 'the', 'wind', 'and', 'through', 'the', 'rain', 'over', 'mountain', 'over', 'plain', 'through', 'the', 'blinding', 'lightning', 'flash', 'and', 'the', 'mighty', 'thunder', 'crash'], ['<sos>', 'ever', 'faithful', 'ever', 'true', 'nothing', 'stops', 'him', 'he', 'll', 'get', 'through', '.'], ['<sos>', 'look', 'out', 'for', 'mr', 'stork', 'that', 'persevering', 'chap'], ['<sos>', 'he', 'll', 'come', 'along', 'and', 'drop', 'a', 'bundle', 'in', 'your', 'lap'], ['<sos>', 'you', 'may', 'be', 'poor', 'or', 'rich', 'it', 'doesn', 't', 'matter', 'which']]
[['through', 'the', 'snow', 'and', 'sleet', 'and', 'hail', 'through'

In [157]:
tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)
encoder_input = pad_sequences(encoder_input, padding="post")

tokenizer_fra = Tokenizer(filters="", lower=False)
tokenizer_fra.fit_on_texts(sents_fra_in)
tokenizer_fra.fit_on_texts(sents_fra_out)

decoder_input = tokenizer_fra.texts_to_sequences(sents_fra_in)
decoder_input = pad_sequences(decoder_input, padding="post")

decoder_target = tokenizer_fra.texts_to_sequences(sents_fra_out)
decoder_target = pad_sequences(decoder_target, padding="post")

In [158]:
print(encoder_input.shape)
print(decoder_input.shape)
print(decoder_target.shape)

(33000, 30)
(33000, 56)
(33000, 56)


In [159]:
vocab_size_en = len(tokenizer_en.word_index) + 1
vocab_size_fra = len(tokenizer_fra.word_index) + 1
print("영어 단어 집합의 크기 (en): {:d}, 프랑스어 단어 집합의 크기 (spa): {:d}".format(vocab_size_en, vocab_size_fra))

영어 단어 집합의 크기 (en): 53949, 프랑스어 단어 집합의 크기 (spa): 11618


In [160]:
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word

tar_to_index = tokenizer_fra.word_index
index_to_tar = tokenizer_fra.index_word

In [161]:
max_src_len = encoder_input.shape[1]
max_tar_len = decoder_input.shape[1]
print('source 문장의 최대 길이 :',max_src_len)
print('target 문장의 최대 길이 :',max_tar_len)

source 문장의 최대 길이 : 30
target 문장의 최대 길이 : 56


In [162]:
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_fra.word_index) + 1

In [163]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print(indices)

[30194 29994 29393 ...   451 16651 32778]


In [164]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [165]:
encoder_input[30997]

array([    1, 51952,  6836,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0], dtype=int32)

In [166]:
decoder_input[30997]

array([   2,    5,  150, 3652,    1,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0], dtype=int32)

In [167]:
decoder_target[30997]

array([   5,  150, 3652,    1,    3,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0], dtype=int32)

In [168]:
n_of_val = int(33000*0.1)
print('검증 데이터의 개수 :',n_of_val)

검증 데이터의 개수 : 3300


In [169]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [170]:
print('훈련 source 데이터의 크기 :',encoder_input_train.shape)
print('훈련 target 데이터의 크기 :',decoder_input_train.shape)
print('훈련 target 레이블의 크기 :',decoder_target_train.shape)
print('테스트 source 데이터의 크기 :',encoder_input_test.shape)
print('테스트 target 데이터의 크기 :',decoder_input_test.shape)
print('테스트 target 레이블의 크기 :',decoder_target_test.shape)

훈련 source 데이터의 크기 : (29700, 30)
훈련 target 데이터의 크기 : (29700, 56)
훈련 target 레이블의 크기 : (29700, 56)
테스트 source 데이터의 크기 : (3300, 30)
테스트 target 데이터의 크기 : (3300, 56)
테스트 target 레이블의 크기 : (3300, 56)


In [171]:
latent_dim = 50

In [172]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [173]:
# 인코더
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(src_vocab_size, latent_dim)(encoder_inputs) # 임베딩 층
enc_masking = Masking(mask_value=0.0)(enc_emb) # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(latent_dim, return_state=True) # 상태값 리턴을 위해 return_state는 True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

In [174]:
# 디코더
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, latent_dim) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True) 

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state=encoder_states)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [175]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [176]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [177]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, None, 50)     2697450     ['input_5[0][0]']                
                                                                                                  
 embedding_3 (Embedding)        (None, None, 50)     580900      ['input_6[0][0]']                
                                                                                            

In [178]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data=([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=128, epochs=50)

Epoch 1/50
233/233 [==============================] - 47s 131ms/step - loss: 2.3719 - acc: 0.8274 - val_loss: 0.9564 - val_acc: 0.8566
Epoch 2/50
233/233 [==============================] - 27s 115ms/step - loss: 0.9178 - acc: 0.8618 - val_loss: 0.8944 - val_acc: 0.8641
Epoch 3/50
233/233 [==============================] - 27s 115ms/step - loss: 0.8802 - acc: 0.8654 - val_loss: 0.8715 - val_acc: 0.8675
Epoch 4/50
233/233 [==============================] - 27s 115ms/step - loss: 0.8527 - acc: 0.8685 - val_loss: 0.8439 - val_acc: 0.8700
Epoch 5/50
233/233 [==============================] - 27s 115ms/step - loss: 0.8263 - acc: 0.8714 - val_loss: 0.8246 - val_acc: 0.8729
Epoch 6/50
233/233 [==============================] - 27s 115ms/step - loss: 0.8054 - acc: 0.8734 - val_loss: 0.8141 - val_acc: 0.8738
Epoch 7/50
233/233 [==============================] - 27s 116ms/step - loss: 0.7882 - acc: 0.8750 - val_loss: 0.7988 - val_acc: 0.8756
Epoch 8/50
233/233 [==============================] - 2

In [179]:
# 인코더
encoder_model = Model(encoder_inputs, encoder_states)

In [180]:
# 디코더 설계 시작
# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2 = dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

In [181]:
# 디코더
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [182]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 정수 생성
  target_seq = np.zeros((1,1))
  target_seq[0, 0] = tar_to_index['<sos>']

  stop_condition = False
  decoded_sentence = ''

  # stop_condition이 True가 될 때까지 루프 반복
  # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 단어로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 단어를 예측 문장에 추가
    decoded_sentence += ' '+sampled_char

    # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
    if (sampled_char == '<eos>' or
        len(decoded_sentence) > 50):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = sampled_token_index

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence

In [183]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_src(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word!=0):
      sentence = sentence + index_to_src[encoded_word] + ' '
  return sentence

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_tar(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word!=0 and encoded_word!=tar_to_index['<sos>'] and encoded_word!=tar_to_index['<eos>']):
      sentence = sentence + index_to_tar[encoded_word] + ' '
  return sentence

In [ ]:
for seq_index in [ 443, 454, 343, 234, 233]:
  input_seq = encoder_input_test[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)
  
  print("입력문장 :",seq_to_src(encoder_input_test[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_test[seq_index]))
  print("번역문장 :",decoded_sentence[1:-5])
  print("-"*50)